# Download OLIDA Excel files into jupyter notebook data folder
On OLIDA website, download Excel files of "Genes", "Diseases", and "Gene Combinations" and upload to a data folder in jupyter notebook 

# Import modules

In [84]:
import pandas as pd

# Get Excel data

In [110]:
# Get gene data
df_gene = pd.read_excel('data/Gene.xlsx', skiprows=1)

# Get disease data
df_disease = pd.read_excel('data/Disease.xlsx', skiprows=1)

#Get gene combo data
df_combo = pd.read_excel('data/GeneCombination.xlsx', skiprows=1)

# Prepare dataframes

In [111]:
#Rename columns for exploding and merging
df_combo.rename(columns = {'Oligogenic variant combinations':'Combinations'}, inplace = True)
df_combo.rename(columns = {'Genes':'Partner Genes'}, inplace = True)
df_disease.rename(columns = {'Disease Name':'Diseases'}, inplace = True)

# Drop unwanted series
df_gene.drop(columns=['Ensembl Id', 'Entrez Id', 'Chromosome', 'Uniprot Accession Number', 'Gene Ontology Molecular Function', 'Essential In Mouse', 'Pathway', 'Variants'], inplace=True)
df_disease.drop(columns = ['Orphanet ID', 'ICD-10 Category', 'Omim Id'], inplace=True)
df_combo.drop(columns=['Entry Id', 'Genes Relationship', 'Protein Interactions', 'Common Pathways', 'Common Pathways', 'GENEmeta'], inplace=True)

# Define function

In [112]:
#Make combos in series into lists then explode
def combo_explode(df):
#     df['Combinations'] = df['Combinations'].str.split(";")
    df['Combinations'] = df['Combinations'].str.replace(';', '').apply(str.split)
    newdf = df.explode('Combinations')
    return newdf.sort_values("Combinations")

 # Execute functions

In [113]:
df_gene = combo_explode(df_gene)
df_disease = combo_explode(df_disease)
df_combo = combo_explode(df_combo)

In [114]:
# Merge dataframes via "Combinations"
mergedf = df_gene.merge(df_disease, on=["Combinations", "Diseases"], how="left")
mergedf2 = mergedf.merge(df_combo, on="Combinations", how="left")

mergedf2.sort_values(by = 'Combinations')

,Gene Name,Combinations,Diseases,ICD-10 ID,Partner Genes
0,CPO,OLI001,Hereditary coproporphyria,E80.2,ALAD; CPO
1,ALAD,OLI001,Hereditary coproporphyria,E80.2,ALAD; CPO
2,NOTCH3,OLI002,Primary ovarian failure; Idiopathic pulmonary ...,NaN,BMPR2; NOTCH3
3,BMPR2,OLI002,Pulmonary arterial hypertension; Heritable pul...,NaN,BMPR2; NOTCH3
4,NOTCH3,OLI003,Primary ovarian failure; Idiopathic pulmonary ...,NaN,BMPR2; NOTCH3
...,...,...,...,...,...
3862,MYH6,OLI998,Hypertrophic cardiomyopathy; Androgen insensit...,NaN,AR; MYH6; NR5A1
3861,NR5A1,OLI998,"46,XY disorder of sex development; Syndrome wi...",NaN,AR; MYH6; NR5A1
3863,AR,OLI998,"Amyotrophic lateral sclerosis; 46,XY disorder ...",NaN,AR; MYH6; NR5A1
3864,SRD5A2,OLI999,Non-syndromic posterior hypospadias; Androgen ...,NaN,AR; SRD5A2


In [116]:
mergedf2.head(10).sort_values(by = 'Combinations')

,Gene Name,Combinations,Diseases,ICD-10 ID,Partner Genes
0,CPO,OLI001,Hereditary coproporphyria,E80.2,ALAD; CPO
1,ALAD,OLI001,Hereditary coproporphyria,E80.2,ALAD; CPO
2,NOTCH3,OLI002,Primary ovarian failure; Idiopathic pulmonary ...,NaN,BMPR2; NOTCH3
3,BMPR2,OLI002,Pulmonary arterial hypertension; Heritable pul...,NaN,BMPR2; NOTCH3
4,NOTCH3,OLI003,Primary ovarian failure; Idiopathic pulmonary ...,NaN,BMPR2; NOTCH3
5,BMPR2,OLI003,Pulmonary arterial hypertension; Heritable pul...,NaN,BMPR2; NOTCH3
6,NOTCH3,OLI004,Primary ovarian failure; Idiopathic pulmonary ...,NaN,ABCC8; NOTCH3
7,ABCC8,OLI004,Pulmonary arterial hypertension associated wit...,NaN,ABCC8; NOTCH3
8,ABCC8,OLI005,Pulmonary arterial hypertension associated wit...,NaN,ABCC8; SARS2
9,SARS2,OLI005,Pulmonary arterial hypertension associated wit...,I27.2,ABCC8; SARS2


# Save to local file

In [115]:
mergedf2.to_csv('olidaexcel.csv', index=False)